In [27]:
import pandas as pd
to=pd.read_csv("/scratch/connectome/3DCNN/data/1.ABCD/4.demo_qc/ABCD_phenotype_total.csv")
ad=pd.read_csv("/scratch/connectome/3DCNN/data/1.ABCD/4.demo_qc/ABCD_ADHD.csv")
con=pd.read_csv("/scratch/connectome/3DCNN/data/1.ABCD/4.demo_qc/ABCD_suicide_control.csv")

In [28]:
pd.read_csv("/scratch/connectome/3DCNN/data/1.ABCD/4.demo_qc/ABCD_ADHD.csv").subjectkey

0       NDARINV00BD7VDC
1       NDARINV00CY2MDM
2       NDARINV00LJVZK2
3       NDARINV00U4FTRU
4       NDARINV0182J779
             ...       
2501    NDARINVZCGL10MP
2502    NDARINVZE3C6X7E
2503    NDARINVZEB043HN
2504    NDARINVZEKA1L7D
2505    NDARINVZER4ED76
Name: subjectkey, Length: 2506, dtype: object

In [29]:
print(ad.shape,con.shape)

(2506, 118) (3108, 45)


In [30]:
import glob
images = glob.glob('/scratch/connectome/3DCNN/data/1.ABCD/3.2.FA_warpped_nii/*')
images_subjectkeys=pd.Series(map(lambda x: x.split("/")[-1].split(".")[0],images),name='subjectkey')
images_subjectkeys[:3]

0    NDARINV89B7M962
1    NDARINV6ZU9NKBV
2    NDARINVLPMG7ZFU
Name: subjectkey, dtype: object

In [31]:
for c in con.columns:
    if 'Attention' in c:print(c)

Unspecified.Attention.Deficit.Hyperactivity.Disorder.x
Attention.Deficit.Hyperactivity.Disorder.x


In [32]:
sjk=pd.concat([ad.subjectkey,con.subjectkey]).reset_index(drop=True)

In [38]:
sjk

0       NDARINV00BD7VDC
1       NDARINV00CY2MDM
2       NDARINV00LJVZK2
3       NDARINV00U4FTRU
4       NDARINV0182J779
             ...       
5609    NDARINVZXPAWB32
5610    NDARINVZZ35KHXP
5611    NDARINVZZ6ZJ2KY
5612    NDARINVZZNX6W2P
5613    NDARINVZZZP87KR
Name: subjectkey, Length: 5614, dtype: object

In [33]:
newdf=to[to.subjectkey.isin(sjk)==True]

In [34]:
newdf.subjectkey.isin(images_subjectkeys).value_counts()

True     3993
False    1621
Name: subjectkey, dtype: int64

In [35]:
images2 = glob.glob('/scratch/connectome/3DCNN/data/1.ABCD/2.sMRI_freesurfer/*')
images_subjectkeys2=pd.Series(map(lambda x: x.split("/")[-1].split(".")[0],images2),name='subjectkey')
images_subjectkeys2[:3]

0    NDARINVBZJGG4AN
1    NDARINVXPZGM0LG
2    NDARINVU9C36KFY
Name: subjectkey, dtype: object

In [36]:
smris=newdf[newdf.subjectkey.isin(images_subjectkeys2)==True]
swithd=smris[smris.subjectkey.isin(images_subjectkeys)==True]
swithd['Attention.Deficit.Hyperactivity.Disorder.x'].value_counts()

0.0    2255
1.0    1736
Name: Attention.Deficit.Hyperactivity.Disorder.x, dtype: int64

In [41]:
pd.merge(pd.merge(ad.subjectkey,to.subjectkey),newdf.subjectkey)

,subjectkey
0,NDARINV00BD7VDC
1,NDARINV00CY2MDM
2,NDARINV00LJVZK2
3,NDARINV00U4FTRU
4,NDARINV0182J779
...,...
2501,NDARINVZCGL10MP
2502,NDARINVZE3C6X7E
2503,NDARINVZEB043HN
2504,NDARINVZEKA1L7D


In [57]:
# revising
ABCD_data_dir = {
    'fmriprep':'/scratch/connectome/3DCNN/data/1.ABCD/1.sMRI_fmriprep/preprocessed_masked/',
    'freesurfer':'/scratch/connectome/3DCNN/data/1.ABCD/2.sMRI_freesurfer/',
    'FA_unwarpped_nii':'/scratch/connectome/3DCNN/data/1.ABCD/3.1.FA_unwarpped_nii/',
    'FA_warpped_nii':'/scratch/connectome/3DCNN/data/1.ABCD/3.2.FA_warpped_nii/',
    'MD_unwarpped_nii':'/scratch/connectome/3DCNN/data/1.ABCD/3.3.MD_unwarpped_nii/',
    'MD_warpped_nii':'/scratch/connectome/3DCNN/data/1.ABCD/3.4.MD_warpped_nii/',
    'RD_unwarpped_nii':'/scratch/connectome/3DCNN/data/1.ABCD/3.5.RD_unwarpped_nii/',
    'RD_warpped_nii':'/scratch/connectome/3DCNN/data/1.ABCD/3.6.RD_warpped_nii/'
}

ABCD_phenotype_dir = {
    'total':'/scratch/connectome/3DCNN/data/1.ABCD/4.demo_qc/ABCD_phenotype_total.csv',
    'ADHD_case':'/scratch/connectome/3DCNN/data/1.ABCD/4.demo_qc/ABCD_ADHD.csv',
    'suicide_case':'/scratch/connectome/3DCNN/data/1.ABCD/4.demo_qc/ABCD_suicide_case.csv',
    'suicide_control':'/scratch/connectome/3DCNN/data/1.ABCD/4.demo_qc/ABCD_suicide_control.csv'
}


    


col_list = ['Attention.Deficit.Hyperactivity.Disorder.x','subjectkey']
dtypes = ['freesurfer','MD_warpped_nii']
### get subject ID and target variables
subject_data = pd.read_csv(ABCD_phenotype_dir['total'])
subject_data = subject_data.loc[:,col_list]

case  = pd.read_csv(ABCD_phenotype_dir['ADHD_case']).subjectkey
control = pd.read_csv(ABCD_phenotype_dir['suicide_control']).subjectkey
filtered_subjectkey = pd.concat([case,control]).reset_index(drop=True)
for data_type in dtypes:
    data_files = glob.glob(ABCD_data_dir[data_type]+'*')
    print(data_files[:3])
    data_subjectkey = pd.Series(map(lambda x: x.split("/")[-1].split(".")[0],data_files),name='subjectkey')
    print(data_subjectkey[:3])
    filtered_subjectkey = pd.merge(filtered_subjectkey,data_subjectkey)
    print(filtered_subjectkey[:3])    

subject_data[subject_data.subjectkey.isin(filtered_subjectkey.subjectkey)==True]

# subject_data = subject_data.sort_values(by='subjectkey')
# subject_data = subject_data.dropna(axis = 0)
# subject_data = subject_data.reset_index(drop=True)

['/scratch/connectome/3DCNN/data/1.ABCD/2.sMRI_freesurfer/NDARINVBZJGG4AN.npy', '/scratch/connectome/3DCNN/data/1.ABCD/2.sMRI_freesurfer/NDARINVXPZGM0LG.npy', '/scratch/connectome/3DCNN/data/1.ABCD/2.sMRI_freesurfer/NDARINVU9C36KFY.npy']
0    NDARINVBZJGG4AN
1    NDARINVXPZGM0LG
2    NDARINVU9C36KFY
Name: subjectkey, dtype: object
        subjectkey
0  NDARINV00BD7VDC
1  NDARINV00CY2MDM
2  NDARINV00LJVZK2
['/scratch/connectome/3DCNN/data/1.ABCD/3.4.MD_warpped_nii/NDARINV89B7M962.nii.gz', '/scratch/connectome/3DCNN/data/1.ABCD/3.4.MD_warpped_nii/NDARINV6ZU9NKBV.nii.gz', '/scratch/connectome/3DCNN/data/1.ABCD/3.4.MD_warpped_nii/NDARINVLPMG7ZFU.nii.gz']
0    NDARINV89B7M962
1    NDARINV6ZU9NKBV
2    NDARINVLPMG7ZFU
Name: subjectkey, dtype: object
        subjectkey
0  NDARINV00BD7VDC
1  NDARINV00CY2MDM
2  NDARINV00LJVZK2


,Attention.Deficit.Hyperactivity.Disorder.x,subjectkey
3,1.0,NDARINV00BD7VDC
4,1.0,NDARINV00CY2MDM
8,1.0,NDARINV00LJVZK2
11,1.0,NDARINV00U4FTRU
15,0.0,NDARINV014RTM1V
...,...,...
10138,0.0,NDARINVV5MHL75K
10142,1.0,NDARINVV5XX9GEF
10148,0.0,NDARINVV6KFJX12
10150,1.0,NDARINVV6MZ4VB1


In [56]:
filtered_subjectkey

,subjectkey
0,NDARINV00BD7VDC
1,NDARINV00CY2MDM
2,NDARINV00LJVZK2
3,NDARINV00U4FTRU
4,NDARINV0182J779
...,...
3986,NDARINVV54EMBM7
3987,NDARINVV56VCMTJ
3988,NDARINVV5MHL75K
3989,NDARINVV6KFJX12


In [55]:
subject_data

,Attention.Deficit.Hyperactivity.Disorder.x,subjectkey
0,0.0,NDARINV003RTV85
1,0.0,NDARINV005V6D2C
2,0.0,NDARINV007W6H7B
3,1.0,NDARINV00BD7VDC
4,1.0,NDARINV00CY2MDM
...,...,...
11870,0.0,NDARINVZZNX6W2P
11871,0.0,NDARINVZZPKBDAC
11872,1.0,NDARINVZZZ2ALR6
11873,1.0,NDARINVZZZNB0XC
